In [1]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [3]:
df_news_clusterizada = pd.read_csv(config["TREATED_ITEMS_CSV"])
# df_news_clusterizada.drop(columns=["Unnamed: 0","Unnamed: 0.1"],inplace=True)

In [4]:
df_news_clusterizada.head(2)

,Unnamed: 0,page,url,issued,modified,title,body,caption
0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad..."
1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....


In [5]:
import numpy as np

# Convert the string column to datetime
df_news_clusterizada["data"] = pd.to_datetime(df_news_clusterizada["issued"])

In [6]:
df_news_clusterizada.head(2)

,Unnamed: 0,page,url,issued,modified,title,body,caption,data
0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",2018-09-13 14:52:55+00:00
1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,2018-05-20 20:42:40+00:00


In [7]:
# Find the most recent date
most_recent_date = df_news_clusterizada["data"].max()

# Calculate the difference in days
df_news_clusterizada["idade_em_dias"] = (most_recent_date - df_news_clusterizada["data"]).dt.days

In [8]:
most_recent_date

Timestamp('2022-08-15 02:57:16+0000', tz='UTC')

In [9]:
df_news_clusterizada["idade_em_dias"].describe()

count    255603.000000
mean        559.549724
std         546.845411
min           0.000000
25%         104.000000
50%         358.000000
75%         909.000000
max        2936.000000
Name: idade_em_dias, dtype: float64

In [10]:
# max_news_age = df_news_clusterizada["idade_em_dias"].max()
max_news_age = 3000
settling_time_taus_factor = 6
time_constant = max_news_age/settling_time_taus_factor
print(max_news_age, time_constant)

3000 500.0


In [11]:
# Assign weights using exponential decay (you can adjust the base for scaling)
df_news_clusterizada["peso"] = np.exp(-df_news_clusterizada["idade_em_dias"] / time_constant) 

In [12]:
# Adjust range to [0.5, 1]
new_min = 0.1
new_max = 1
df_news_clusterizada["peso_ajustado"] = df_news_clusterizada["peso"] * (new_max - new_min) + new_min


In [13]:
df_news_clusterizada.head(2)

,Unnamed: 0,page,url,issued,modified,title,body,caption,data,idade_em_dias,peso,peso_ajustado
0,0,7371a9b5-5824-4c57-8704-00a74feebe79,http://g1.globo.com/al/alagoas/noticia/2018/09...,2018-09-13 14:52:55+00:00,2018-09-14 16:14:49+00:00,Corpo de motorista da Uber é encontrado em can...,Corpo de motorista de aplicativo desaparecido ...,"Segundo a polícia, Antônio Vitor foi solicitad...",2018-09-13 14:52:55+00:00,1431,0.057154,0.151439
1,1,7a5ea08f-4583-49e2-ba52-a71999443f7b,http://g1.globo.com/am/amazonas/noticia/detent...,2018-05-20 20:42:40+00:00,2018-05-20 20:42:40+00:00,Detento recapturado após fuga por túnel volta ...,Detento disse que passou nome falso ao dar ent...,Ele tinha registro em presídio com nome falso....,2018-05-20 20:42:40+00:00,1547,0.045320,0.140788


In [14]:
df_news_clusterizada.describe()

,Unnamed: 0,idade_em_dias,peso,peso_ajustado
count,255603.000000,255603.000000,255603.000000,255603.000000
mean,127801.000000,559.549724,0.492212,0.542991
std,73786.374765,546.845411,0.329121,0.296209
min,0.000000,0.000000,0.002817,0.102536
25%,63900.500000,104.000000,0.162350,0.246115
50%,127801.000000,358.000000,0.488703,0.539833
75%,191701.500000,909.000000,0.812207,0.830986
max,255602.000000,2936.000000,1.000000,1.000000


In [15]:
df_news_clusterizada.to_csv(config["DF_ITEMS_CLUSTERED_FEATURES_ADJ"])